In [ ]:
#| default_exp 93-ngame-ep-for-wikititles-with-input-concatenation-1-0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os,torch, torch.multiprocessing as mp, pickle, numpy as np
from xcai.basics import *
from xcai.models.PPP0XX import DBT009,DBT011

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
os.environ['WANDB_MODE'] = 'disabled'

In [ ]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT']='xc-nlg_93-ngame-ep-for-wikititles'

### Load data

In [ ]:
data_dir = '/home/scai/phd/aiz218323/Projects/XC_NLG/data'

In [ ]:
block = XCBlock.from_cfg(data_dir, 'data_metas', dset='wikititles', valid_pct=0.001, tfm='xcnlg', tokenizer='distilbert-base-uncased', 
                         smp_features=[('lbl2data',1,1)])

/home/scai/phd/aiz218323/.local/lib/python3.9/site-packages/xclib-0.97-py3.9-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikititles_data-metas_distilbert-base-uncased_rm_ngame-hlk-sal.pkl'

In [ ]:
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

### Augmentation code

In [ ]:
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikititles_data-metas_distilbert-base-uncased_rm_ngame-hlk-sal.pkl'
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 'data', 128, True)
block = AugmentMetaInputIdsTfm.apply(block, 'hlk_meta', 'lbl', 128, True)

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikititles_data-metas_distilbert-base-uncased_rm_ngame-hlk-sal-128.pkl'
with open(pkl_file, 'wb') as file: pickle.dump(block, file)

In [ ]:
pkl_file = f'{pkl_dir}/processed/wikititles_data-metas_distilbert-base-uncased_rm_ngame-hlk-sal-128.pkl'
with open(pkl_file, 'rb') as file: block = pickle.load(file)

### Augmentation data

In [ ]:
#| export
pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets'
pkl_file = f'{pkl_dir}/processed/wikititles_data-metas_distilbert-base-uncased_rm_ngame-hlk-sal-128.pkl'
with open(pkl_file, 'rb') as file: block = pickle.load(file)

In [ ]:
block.train.dset.data.data_info['input_ids_org'] = block.train.dset.data.data_info['input_ids']
block.train.dset.data.data_info['attention_mask_org'] = block.train.dset.data.data_info['attention_mask']
block.test.dset.data.data_info['input_ids_org'] = block.test.dset.data.data_info['input_ids']
block.test.dset.data.data_info['attention_mask_org'] = block.test.dset.data.data_info['attention_mask']

block.train.dset.data.lbl_info['input_ids_org'] = block.train.dset.data.lbl_info['input_ids']
block.train.dset.data.lbl_info['attention_mask_org'] = block.train.dset.data.lbl_info['attention_mask']
block.test.dset.data.lbl_info['input_ids_org'] = block.test.dset.data.lbl_info['input_ids']
block.test.dset.data.lbl_info['attention_mask_org'] = block.test.dset.data.lbl_info['attention_mask']

In [ ]:
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_org']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_org']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_org']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_org']

block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_org']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_org']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_org']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_org']

In [ ]:
#| export
block.train.dset.data.data_info['input_ids'] = block.train.dset.data.data_info['input_ids_aug_hlk']
block.train.dset.data.data_info['attention_mask'] = block.train.dset.data.data_info['attention_mask_aug_hlk']
block.test.dset.data.data_info['input_ids'] = block.test.dset.data.data_info['input_ids_aug_hlk']
block.test.dset.data.data_info['attention_mask'] = block.test.dset.data.data_info['attention_mask_aug_hlk']

In [ ]:
#| export
block.train.dset.data.lbl_info['input_ids'] = block.train.dset.data.lbl_info['input_ids_aug_hlk']
block.train.dset.data.lbl_info['attention_mask'] = block.train.dset.data.lbl_info['attention_mask_aug_hlk']
block.test.dset.data.lbl_info['input_ids'] = block.test.dset.data.lbl_info['input_ids_aug_hlk']
block.test.dset.data.lbl_info['attention_mask'] = block.test.dset.data.lbl_info['attention_mask_aug_hlk']

In [ ]:
print(np.mean([len(o) for o in block.train.dset.data.lbl_info['input_ids_aug_hlk']]))
print(np.mean([len(o) for o in block.train.dset.data.lbl_info['attention_mask_aug_hlk']]))

7.331985550921029
7.331985550921029


In [ ]:
print(np.mean([len(o) for o in block.train.dset.data.data_info['input_ids_aug_hlk']]))
print(np.mean([len(o) for o in block.train.dset.data.data_info['attention_mask_aug_hlk']]))

48.80174766500992
48.80174766500992


## Training

In [ ]:
#| export
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/93-ngame-ep-for-wikititles-with-input-concatenation-1-0',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
)

In [ ]:
#| export
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1, 
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)
model.init_dr_head()

In [ ]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [ ]:
learn.train()

In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()

## Prediction

In [ ]:
args = XCLearningArguments(
    output_dir='/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    predict_with_representation=True,
    representation_search_type='BRUTEFORCE',
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    use_encoder_parallel=True,
    fp16=True,
)

In [ ]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [ ]:
output_dir = f"/home/scai/phd/aiz218323/scratch/outputs/{os.path.basename(args.output_dir)}"
mname = f'{output_dir}/{os.path.basename(get_best_model(output_dir))}'

In [ ]:
mname

'/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2/checkpoint-85000'

In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT009.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', bsz=bsz, tn_targ=5000, margin=0.3, tau=0.1,
                               n_negatives=10, apply_softmax=True, use_encoder_parallel=True)

Some weights of DBT009 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.dr_layer_norm.bias', 'encoder.dr_layer_norm.weight', 'encoder.dr_projector.bias', 'encoder.dr_projector.weight', 'encoder.dr_transform.bias', 'encoder.dr_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from safetensors import safe_open

model_weight_file = f'{mname}/model.safetensors'

model_weights = {}
with safe_open(model_weight_file, framework="pt") as file:
    for k in file.keys(): model_weights[k] = file.get_tensor(k)
        

In [ ]:
model.load_state_dict(model_weights, strict=False)

_IncompatibleKeys(missing_keys=['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ffn.lin2.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.

In [ ]:
learn = XCLearner(
    model=model, 
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
o = learn.predict(block.test.dset)

  0%|          | 0/196 [00:00<?, ?it/s]

/scratch/scai/phd/aiz218323/anaconda3/envs/xc_nlg/lib/python3.9/site-packages/scipy/sparse/_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,24.1771,16.1361,12.2884,7.9308,24.1771,24.4573,25.4609,27.2161,20.1098,21.5063,23.1904,26.8649,20.1098,22.0686,23.477,25.37,31.8421,46.4144,49.9468,0.0294,262.9244,675.156,0.422


In [ ]:
mname

'/home/scai/phd/aiz218323/scratch/outputs/67-ngame-ep-for-wikiseealso-with-input-concatenation-1-2/checkpoint-85000'

In [ ]:
pred_dir = f"{mname}/predictions/"
os.makedirs(pred_dir, exist_ok=True)

with open(f'{pred_dir}/test_predictions.pkl', 'wb') as file: pickle.dump(o, file)